<a href="https://colab.research.google.com/github/Brennan-Richards/Deep-Learning-R-and-D/blob/main/Graph_NN_Protein_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PURPOSE: Learn to create GNNs (in the language of the Spektral library).

CONTENTS: Contents pertaining to the tutorial in the [Spektral - Getting Started documentation](https://graphneural.network/getting-started/), a step-by-step walkthrough for a graph-level prediction task.

In [ ]:
!pip install spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 KB 7.3 MB/s eta 0:00:00


In [ ]:
import numpy as np

Load dataset and explore possible operations

In [ ]:
from spektral.datasets import TUDataset
dataset = TUDataset('PROTEINS')

dataset

100%|████████████████████████████████████████| 447k/447k [00:00<00:00, 1.02MB/s]


Successfully loaded PROTEINS.


TUDataset(n_graphs=1113)

In [ ]:
# Index data set
dataset[0]

Graph(n_nodes=98, n_node_features=4, n_edge_features=None, n_labels=2)

In [ ]:
# Shuffle graphs in data set
np.random.shuffle(dataset)
dataset[0]

<ipython-input-9-cf07d3aca7f9>:2: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(dataset)


Graph(n_nodes=23, n_node_features=4, n_edge_features=None, n_labels=2)

In [ ]:
# Slice data set
part = dataset[:10]
part

TUDataset(n_graphs=10)

Dataset transformations

In [ ]:
# Transform the dataset inplace by removing graphs with more than 500 nodes
dataset.filter(lambda g: g.n_nodes < 500)
dataset

TUDataset(n_graphs=1111)

In [ ]:
# Apply more transforms

max_degree = int(dataset.map(lambda g: g.a.sum(-1).max(), reduce=max)) # Compute the max degree of the data set = size of one-hot encoding vector

print(max_degree)

from spektral.transforms import Degree

dataset.apply(Degree(max_degree))

12


In [ ]:
# NOTE: max_degree + 1 new node features were added.
dataset[0]

Graph(n_nodes=23, n_node_features=17, n_edge_features=None, n_labels=2)

In [ ]:
# "Since we will be using a GCNConv layer in our GNN, we also want to follow the original paper (https://arxiv.org/abs/1609.02907) that introduced this layer and do some extra pre-processing of the adjacency matrix."
from spektral.transforms import GCNFilter

dataset.apply(GCNFilter())

# NOTE: "If you don't want to go back to the literature every time, every convolutional layer in Spektral has a preprocess(a) method that you can use to transform the adjacency matrix as needed."

Create train and test data sets

In [ ]:
train_dataset = dataset[:int(len(dataset)*0.8)] # The first 80%
print(len(train_dataset))

888


In [ ]:
test_dataset = dataset[int(len(dataset)*0.8):] # The rest beyond the train set's 80%
print(len(test_dataset))

223


Build and train a GNN

In [ ]:
# "Since Spektral is designed as an extension of Keras, you can plug any Spektral layer into a Keras Model without modifications". Must use functional API.
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [ ]:
class GNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

In [ ]:
model = GNN(32, dataset.n_labels)
model.compile('adam', 'categorical_crossentropy')

In [ ]:
"""
Unlike regular data, like images or sequences, graphs cannot be stretched, cut, or reshaped so that we can fit them into tensors of pre-defined shapes.
If a graph has 10 nodes and another one has 4, we have to keep them that way. This means that iterating over a dataset in mini-batches is not trivial
and we cannot simply use the model.fit() method of Keras as-is.

We have to use a data Loader (read: https://graphneural.network/data-modes/).
"""

In [ ]:
from spektral.data import BatchLoader

train_loader = BatchLoader(train_dataset, batch_size=32)

In [ ]:
model.fit(train_loader.load(), steps_per_epoch=train_loader.steps_per_epoch, epochs=10)

Epoch 1/10
 6/28 [=====>........................] - ETA: 0s - loss: 5.4338

/usr/local/lib/python3.8/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


28/28 [==============================] - 0s 12ms/step - loss: 4.8000
Epoch 2/10
28/28 [==============================] - 1s 19ms/step - loss: 4.7673
Epoch 3/10
28/28 [==============================] - 0s 17ms/step - loss: 4.3431
Epoch 4/10
28/28 [==============================] - 0s 17ms/step - loss: 4.3096
Epoch 5/10
28/28 [==============================] - 0s 18ms/step - loss: 4.2121
Epoch 6/10
28/28 [==============================] - 0s 12ms/step - loss: 4.6701
Epoch 7/10
28/28 [==============================] - 0s 12ms/step - loss: 5.3525
Epoch 8/10
28/28 [==============================] - 0s 11ms/step - loss: 3.7384
Epoch 9/10
28/28 [==============================] - 0s 13ms/step - loss: 3.2373
Epoch 10/10
28/28 [==============================] - 0s 11ms/step - loss: 3.5623


Evaluate

In [ ]:
test_loader = BatchLoader(test_dataset, batch_size=32)


In [ ]:
loss = model.evaluate(test_loader.load(), steps=test_loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

7/7 [==============================] - 0s 6ms/step - loss: 1.6038
Test loss: 1.6037871837615967


Moving forward



*   [Data modes reading](https://graphneural.network/data-modes/)
*   [Examples!](https://graphneural.network/examples/)

